In [ ]:
# DO NOT CHANGE ANYTHING IN THIS CELL
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tensorflow.keras
import time
import pytz
import imutils
import hashlib
import pickle
from datetime import datetime

# scikit learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# tensorflow libraries
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Dense, Activation, Dropout, Flatten, Input
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2

from typing import List
from pathlib import Path
from imutils.video import VideoStream
from PIL import Image

label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:

npy_data = 'dataset/numpy_images.npy'
label_data = 'dataset/labels.pkl'
if Path(npy_data).is_file() and Path(label_data).is_file():
    print('Loading from previously processed training data X and labels Y')
    with open(npy_data, 'rb') as f:
        X = np.load(f)
    with open(label_data, 'rb') as f:
        Y = pickle.load(f)
        labels = [label_map[e] for e in Y]
else:
    print('No training numpy array saved previously. Please run the preprocessing notebook.')


assert X.shape == (32298, 48, 48, 1)
print(f'Validated that X.shape is (32298, 48, 48, 1)')

assert len(Y) == X.shape[0]
print(f'Validated that len(Y) is {len(Y)}')

Loading from previously processed training data X and labels Y
Validated that X.shape is (32298, 48, 48, 1)
Validated that len(Y) is 32298


In [ ]:
# your code here
from keras.utils import to_categorical
print(Y.shape)
Y = to_categorical(Y)
print(Y.shape)
X_train, X_val, y_train, y_val = train_test_split(X, Y,test_size=0.1, random_state=1234)

Using TensorFlow backend.


(32298,)
(32298, 7)


In [ ]:
# your code here
def generate_id(digest_size=4):
    hash = hashlib.blake2b(digest_size=digest_size)
    hash.update(str(time.time()).encode('utf-8'))
    return hash.hexdigest()

In [ ]:
def simple_convnet():
    model = Sequential([Conv2D(64, (8,8), padding='same', activation='relu', input_shape=(48,48,1)),
                      BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(128, (8,8), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(256, (8,8), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(7,activation = 'softmax')
  ])
    return model

model = simple_convnet()
model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


In [ ]:
# check the summary of your model (you don't have to do anything in this cell)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 48, 48, 64)        4160      
_________________________________________________________________
batch_normalization_9 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 128)       524416    
_________________________________________________________________
batch_normalization_10 (Batc (None, 24, 24, 128)       512       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 128)      

In [ ]:
PATH ='/content/dataset'
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
batch_size = 128
epochs = 15
train_image_generator = ImageDataGenerator(
                    rotation_range=10,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    #horizontal_flip=True,
                    zoom_range=0.1
)
validation_image_generator = ImageDataGenerator()
train_data_gen = train_image_generator.flow(X_train ,y_train,batch_size = batch_size)

val_data_gen = validation_image_generator.flow(X_val ,y_val,batch_size = batch_size)


In [ ]:
# DO NOT CHANGE ANYTHING IN THIS CELL
utc_now = pytz.utc.localize(datetime.utcnow())
date_today = utc_now.astimezone(pytz.timezone('Asia/Shanghai')).strftime('%Y-%m-%d')
#path_model=f'/content/drive/My Drive/checkpoints/convnet/{date_today}/' + '{epoch:02d}-{val_loss:.6f}.hdf5'
#Path(f'/content/drive/My Drive/checkpoints/convnet/{date_today}').mkdir(parents=True, exist_ok=True)
path_model=f'checkpoints/convnet/{date_today}/' + '{epoch:02d}-{val_loss:.6f}.hdf5'
Path(f'checkpoints/convnet/{date_today}').mkdir(parents=True, exist_ok=True)


In [ ]:
# implement training of the model
# your code here
model.summary()
history = model.fit(
    train_data_gen,
    epochs=15,
    batch_size= batch_size,
    validation_data=val_data_gen,
    callbacks=[
        ModelCheckpoint(filepath=path_model),
        EarlyStopping(patience=15),
        ReduceLROnPlateau(patience=6, factor=0.3)
    ]
)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 48, 48, 64)        4160      
_________________________________________________________________
batch_normalization_9 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 128)       524416    
_________________________________________________________________
batch_normalization_10 (Batc (None, 24, 24, 128)       512       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 128)      

In [ ]:
model.save('checkpoints/convne/checkpoint1.hdf5')
print(path_model)